# UniFi Data Analysis

Fetch data from UniFi Site Manager API and create DataFrames for analysis.

**💡 Debugging Tip:** For better visualization viewing, run the "Launch Notebook in Browser" cell at the end to open this notebook in a browser with full automation hooks!


In [1]:
# Install required libraries
import sys
!{sys.executable} -m pip install requests pandas python-dotenv --quiet


In [2]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables from ~/.env
env_path = Path.home() / '.env'
load_dotenv(env_path)

# Get API key from environment
api_key = os.getenv('UNIFI_API_KEY')

if not api_key:
    raise ValueError("UNIFI_API_KEY not found in ~/.env file. Please add it.")

print(f"✓ API key loaded from ~/.env")
print(f"✓ Ready to fetch data from UniFi Site Manager API")

# Create API session
api_session = requests.Session()
api_session.headers.update({
    'X-API-Key': api_key,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
})


✓ API key loaded from ~/.env
✓ Ready to fetch data from UniFi Site Manager API


## Fetch Data from UniFi API


In [ ]:
# UniFi Site Manager API endpoints
endpoints = {
    'hosts': 'https://api.ui.com/v1/hosts',
    'sites': 'https://api.ui.com/v1/sites',
    'devices': 'https://api.ui.com/v1/devices',
    'networks': 'https://api.ui.com/v1/networks',
    'clients': 'https://api.ui.com/v1/clients',
    'events': 'https://api.ui.com/v1/events',
    'system': 'https://api.ui.com/v1/system',
}

print("Fetching data from UniFi Site Manager API...\n")
api_data = {}

for name, endpoint in endpoints.items():
    try:
        response = api_session.get(endpoint, timeout=15)
        if response.status_code == 200:
            data = response.json()
            api_data[name] = data
            if isinstance(data, dict) and 'data' in data:
                count = len(data['data']) if isinstance(data['data'], list) else 'N/A'
                print(f"✓ {name:12s} - {endpoint.split('/')[-1]:20s} - {count} items")
            else:
                print(f"✓ {name:12s} - {endpoint.split('/')[-1]:20s} - data retrieved")
        elif response.status_code == 401:
            print(f"✗ {name:12s} - Unauthorized (check API key)")
        elif response.status_code == 403:
            print(f"✗ {name:12s} - Forbidden (insufficient permissions)")
        else:
            print(f"✗ {name:12s} - Error {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"✗ {name:12s} - Connection error: {e}")

print(f"\n✓ Fetched data from {len(api_data)} endpoints")


Fetching data from UniFi Site Manager API...

✓ hosts        - hosts                - 2 items
✓ sites        - sites                - 2 items
✓ devices      - devices              - 1 items
✗ networks     - Error 404
✗ clients      - Error 404
✗ events       - Error 404
✗ system       - Error 404

✓ Fetched data from 3 endpoints


In [ ]:
def create_dataframe(data, name="data"):
    """
    Convert API response data into a pandas DataFrame using json_normalize
    for proper handling of nested JSON structures
    
    Args:
        data: API response data (dict with 'data' key, or list, or dict)
        name: Name for the DataFrame (for display purposes)
    """
    if isinstance(data, dict):
        if 'data' in data:
            items = data['data']
        else:
            items = [data]
    elif isinstance(data, list):
        items = data
    else:
        return None
    
    if not items or len(items) == 0:
        return None
    
    if isinstance(items, list) and len(items) > 0:
        try:
            # Use json_normalize for nested structures
            df = pd.json_normalize(items)
            return df
        except Exception as e:
            print(f"✗ Error creating DataFrame for {name}: {e}")
            import traceback
            traceback.print_exc()
            # Fallback to regular DataFrame
            try:
                df = pd.DataFrame(items)
                return df
            except Exception as e2:
                print(f"✗ Fallback also failed for {name}: {e2}")
                return None
    else:
        return None

# Create DataFrames from API data
print("Creating DataFrames...\n")
dataframes = {}

for name, data in api_data.items():
    df = create_dataframe(data, name)
    if df is not None:
        dataframes[name] = df
        print(f"✓ {name:12s} - {df.shape[0]:4d} rows × {df.shape[1]:3d} columns")

print(f"\n✓ Created {len(dataframes)} DataFrame(s)")
print(f"\nAvailable DataFrames: {list(dataframes.keys())}")


Creating DataFrames...

✓ hosts        -    2 rows × 151 columns
✓ sites        -    2 rows ×  43 columns
✓ devices      -    1 rows ×   3 columns

✓ Created 3 DataFrame(s)

Available DataFrames: ['hosts', 'sites', 'devices']


## Display DataFrame Summaries


In [5]:
# Display summary of each DataFrame
for name, df in dataframes.items():
    print("="*60)
    print(f"DATAFRAME: {name.upper()}")
    print("="*60)
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"\nColumns ({len(df.columns)}):")
    print(", ".join(df.columns.tolist()[:15]))
    if len(df.columns) > 15:
        print(f"... and {len(df.columns) - 15} more")
    print(f"\nFirst few rows:")
    print(df.head())
    print()


DATAFRAME: HOSTS
Shape: 2 rows × 151 columns

Columns (151):
id, hardwareId, type, ipAddress, owner, isBlocked, registrationTime, lastConnectionStateChange, latestBackupTime, userData.permissions.network.management, userData.status, reportedState.controller_uuid, reportedState.firmware_version, reportedState.hardware_id, reportedState.host_type
... and 136 more

First few rows:
                                                  id  \
0               67be1762-80ee-46dd-a25f-dd69d18da8c6   
1  74ACB93D0FFB0000000004AEDD420000000004E4063400...   

                             hardwareId            type     ipAddress  owner  \
0  0ad4313b-ebc9-40e9-b5fd-f020e99c15e9  network-server  73.78.113.14   True   
1  5883cf3e-f791-5bb7-815f-fb78f4fde4d0         console  73.78.113.14   True   

   isBlocked      registrationTime lastConnectionStateChange  \
0      False  2024-09-20T00:26:41Z      2025-09-08T22:14:16Z   
1      False                            2025-10-23T22:31:50Z   

       latestBac

## Access Individual DataFrames

All DataFrames are stored in the `dataframes` dictionary. Access them like:

- `dataframes['hosts']` - Hosts/sites information
- `dataframes['devices']` - Network devices
- `dataframes['clients']` - Connected clients
- `dataframes['networks']` - Network configurations
- `dataframes['events']` - System events
- `dataframes['system']` - System information

### Quick Access Examples


In [6]:
# Quick access examples

# Show available DataFrames
print("Available DataFrames:")
for name in dataframes.keys():
    df = dataframes[name]
    print(f"  - {name}: {df.shape[0]} rows, {df.shape[1]} columns")

# Example: Display hosts/sites
if 'hosts' in dataframes:
    print("\n" + "="*60)
    print("HOSTS/SITES")
    print("="*60)
    print(dataframes['hosts'])
elif 'sites' in dataframes:
    print("\n" + "="*60)
    print("SITES")
    print("="*60)
    print(dataframes['sites'])

# Example: Display devices
if 'devices' in dataframes:
    print("\n" + "="*60)
    print("DEVICES")
    print("="*60)
    # Show key columns if they exist
    key_cols = ['name', 'model', 'ip', 'mac', 'version', 'status']
    available_cols = [col for col in key_cols if col in dataframes['devices'].columns]
    if available_cols:
        print(dataframes['devices'][available_cols].head(10))
    else:
        print(dataframes['devices'].head(10))

# Example: Display clients
if 'clients' in dataframes:
    print("\n" + "="*60)
    print("CLIENTS")
    print("="*60)
    key_cols = ['hostname', 'ip', 'mac', 'essid', 'name']
    available_cols = [col for col in key_cols if col in dataframes['clients'].columns]
    if available_cols:
        print(dataframes['clients'][available_cols].head(10))
    else:
        print(dataframes['clients'].head(10))


Available DataFrames:
  - hosts: 2 rows, 151 columns
  - sites: 2 rows, 43 columns
  - devices: 1 rows, 3 columns

HOSTS/SITES
                                                  id  \
0               67be1762-80ee-46dd-a25f-dd69d18da8c6   
1  74ACB93D0FFB0000000004AEDD420000000004E4063400...   

                             hardwareId            type     ipAddress  owner  \
0  0ad4313b-ebc9-40e9-b5fd-f020e99c15e9  network-server  73.78.113.14   True   
1  5883cf3e-f791-5bb7-815f-fb78f4fde4d0         console  73.78.113.14   True   

   isBlocked      registrationTime lastConnectionStateChange  \
0      False  2024-09-20T00:26:41Z      2025-09-08T22:14:16Z   
1      False                            2025-10-23T22:31:50Z   

       latestBackupTime userData.permissions.network.management  ...  \
0                                                       [admin]  ...   
1  2025-10-31T09:31:45Z                                 [admin]  ...   

                reportedState.uidb.guid              

## Export DataFrames to CSV

Export any DataFrame to CSV for further analysis or sharing.


In [7]:
# Export all DataFrames to CSV
import os
from datetime import datetime

# Create output directory
output_dir = "unifi_data_export"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print(f"Exporting DataFrames to '{output_dir}' directory...\n")

for name, df in dataframes.items():
    filename = f"{output_dir}/unifi_{name}_{timestamp}.csv"
    df.to_csv(filename, index=False)
    print(f"✓ Exported {name:12s} → {filename}")

print(f"\n✓ Exported {len(dataframes)} DataFrames")
print(f"Files saved in: {os.path.abspath(output_dir)}")

# You can also export individual DataFrames:
# dataframes['hosts'].to_csv('hosts.csv', index=False)
# dataframes['devices'].to_csv('devices.csv', index=False)


Exporting DataFrames to 'unifi_data_export' directory...

✓ Exported hosts        → unifi_data_export/unifi_hosts_20251102_201520.csv
✓ Exported sites        → unifi_data_export/unifi_sites_20251102_201520.csv
✓ Exported devices      → unifi_data_export/unifi_devices_20251102_201520.csv

✓ Exported 3 DataFrames
Files saved in: /Users/lou/Downloads/mid-linux-container-recipe.zurich-07-01-2025__patch2-09-24-2025_10-12-2025_0904.linux.x86-64/unifi_data_export


## Analysis Examples

Some common analysis operations you can perform on the DataFrames.


In [8]:
# Analysis examples

# 1. Count devices by model
if 'devices' in dataframes:
    if 'model' in dataframes['devices'].columns:
        print("Device count by model:")
        print(dataframes['devices']['model'].value_counts())
        print()

# 2. Count clients by SSID
if 'clients' in dataframes:
    if 'essid' in dataframes['clients'].columns:
        print("Clients by SSID:")
        print(dataframes['clients']['essid'].value_counts())
        print()

# 3. Show online vs offline devices
if 'devices' in dataframes:
    if 'state' in dataframes['devices'].columns:
        print("Device status:")
        print(dataframes['devices']['state'].value_counts())
        print()
    elif 'status' in dataframes['devices'].columns:
        print("Device status:")
        print(dataframes['devices']['status'].value_counts())
        print()

# 4. Network information
if 'networks' in dataframes:
    print("Networks:")
    key_cols = ['name', 'purpose', 'ip_subnet', 'enabled']
    available_cols = [col for col in key_cols if col in dataframes['networks'].columns]
    if available_cols:
        print(dataframes['networks'][available_cols])
    else:
        print(dataframes['networks'].head())

print("\n💡 Tip: Explore the DataFrames using pandas operations!")
print("   Examples:")
print("   - dataframes['devices'].describe()")
print("   - dataframes['clients'].groupby('essid').size()")
print("   - dataframes['devices'].filter(like='name')")



💡 Tip: Explore the DataFrames using pandas operations!
   Examples:
   - dataframes['devices'].describe()
   - dataframes['clients'].groupby('essid').size()
   - dataframes['devices'].filter(like='name')


## Save Data for Schema Analysis

Save the DataFrames so they can be loaded in the schema/database notebook.


In [ ]:
# Save DataFrames for reuse in schema notebook
import pickle
from pathlib import Path

data_dir = Path("unifi_data_cache")
data_dir.mkdir(exist_ok=True)

# Save DataFrames as pickle (preserves all data types and nested structures)
dataframes_file = data_dir / "dataframes.pkl"
with open(dataframes_file, 'wb') as f:
    pickle.dump(dataframes, f)

# Also save as parquet for each DataFrame (more portable, but may lose nested structures)
parquet_dir = data_dir / "parquet"
parquet_dir.mkdir(exist_ok=True)

for name, df in dataframes.items():
    parquet_file = parquet_dir / f"{name}.parquet"
    try:
        # Try to save as parquet (won't work for columns with lists/dicts)
        df.to_parquet(parquet_file, index=False)
    except Exception as e:
        # If parquet fails, save as pickle for this specific dataframe
        pickle_file = parquet_dir / f"{name}.pkl"
        with open(pickle_file, 'wb') as f:
            pickle.dump(df, f)

print(f"✓ DataFrames saved:")
print(f"   - {dataframes_file} (all DataFrames as pickle)")
print(f"   - {parquet_dir}/ (individual files)")
print(f"\n💾 To load in another notebook:")
print(f"   import pickle")
print(f"   with open('{dataframes_file}', 'rb') as f:")
print(f"       dataframes = pickle.load(f)")


✓ DataFrames saved:
   - unifi_data_cache/dataframes.pkl (all DataFrames as pickle)
   - unifi_data_cache/parquet/ (individual files)

💾 To load in another notebook:
   import pickle
   with open('unifi_data_cache/dataframes.pkl', 'rb') as f:
       dataframes = pickle.load(f)


## Management APIs Discovery

Searching for management/configuration APIs for DNS, WAN/HA, VPN, etc.

**Note:** According to the [official Site Manager API documentation](https://developer.ui.com/site-manager-api/gettingstarted), the API is currently **read-only**. Future versions will include write operations and more granular configuration management.

**Official Endpoints from Documentation:**
- List Hosts
- Get Host by ID
- List Sites
- List Devices
- Get ISP Metrics (for monitoring internet providers)
- Query ISP Metrics
- **List SD-WAN Configs** (for WAN/HA configuration)
- Get SD-WAN Config by ID
- Get SD-WAN Config Status


In [ ]:
# Test official Site Manager API endpoints from documentation
print("="*60)
print("OFFICIAL SITE MANAGER API ENDPOINTS")
print("="*60)
print("Per documentation: https://developer.ui.com/site-manager-api/gettingstarted\n")

# Get host and site IDs from existing data
if 'hosts' in dataframes:
    df_hosts = dataframes['hosts']
    host_ids = df_hosts['id'].dropna().unique().tolist() if 'id' in df_hosts.columns else []
    host_id = host_ids[0] if host_ids else None
else:
    host_id = None

if 'sites' in dataframes:
    df_sites = dataframes['sites']
    site_ids = df_sites['siteId'].dropna().unique().tolist() if 'siteId' in df_sites.columns else []
    site_id = site_ids[0] if site_ids else None
else:
    site_id = None

# Official endpoints from documentation
official_endpoints = {
    'List Hosts': 'https://api.ui.com/v1/hosts',
    'Get Host by ID': f'https://api.ui.com/v1/hosts/{host_id}' if host_id else None,
    'List Sites': 'https://api.ui.com/v1/sites',
    'List Devices': 'https://api.ui.com/v1/devices',
    'Get ISP Metrics': 'https://api.ui.com/v1/isp-metrics',
    'Query ISP Metrics': 'https://api.ui.com/v1/isp-metrics/query',
    'List SD-WAN Configs': 'https://api.ui.com/v1/sd-wan-configs',
}

found_endpoints = {}

print("Testing official endpoints:\n")
for name, endpoint in official_endpoints.items():
    if endpoint is None:
        continue
    
    try:
        resp = api_session.get(endpoint, timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            found_endpoints[name] = endpoint
            
            if isinstance(data, dict):
                if 'data' in data:
                    items = data['data']
                    if isinstance(items, list):
                        print(f"✓ {name:30s}: {len(items)} items")
                        if len(items) > 0 and isinstance(items[0], dict):
                            print(f"  → Keys: {list(items[0].keys())[:8]}")
                    else:
                        print(f"✓ {name:30s}: Config - {list(data.keys())[:5]}")
                else:
                    print(f"✓ {name:30s}: Available - {list(data.keys())[:5]}")
            elif isinstance(data, list):
                print(f"✓ {name:30s}: {len(data)} items")
        elif resp.status_code == 404:
            print(f"? {name:30s}: Not found (may need parameters)")
        elif resp.status_code == 400:
            print(f"? {name:30s}: Bad request (may need query parameters)")
        else:
            print(f"? {name:30s}: {resp.status_code}")
    except Exception as e:
        print(f"✗ {name:30s}: {str(e)[:40]}")

print(f"\n✓ Found {len(found_endpoints)} accessible endpoints")

# If SD-WAN configs found, get details
if 'List SD-WAN Configs' in found_endpoints:
    print("\n" + "="*60)
    print("SD-WAN CONFIGURATION (WAN/HA)")
    print("="*60)
    
    sdwan_resp = api_session.get('https://api.ui.com/v1/sd-wan-configs', timeout=15)
    if sdwan_resp.status_code == 200:
        sdwan_data = sdwan_resp.json()
        if 'data' in sdwan_data and isinstance(sdwan_data['data'], list):
            configs = sdwan_data['data']
            if len(configs) > 0:
                print(f"\nFound {len(configs)} SD-WAN configuration(s):\n")
                for i, config in enumerate(configs):
                    print(f"SD-WAN Config {i+1}:")
                    for key, value in config.items():
                        if isinstance(value, (dict, list)):
                            print(f"  {key}: {type(value).__name__}")
                            if isinstance(value, dict):
                                print(f"    → {list(value.keys())[:10]}")
                            elif isinstance(value, list) and len(value) > 0:
                                print(f"    → {len(value)} items")
                                if isinstance(value[0], dict):
                                    print(f"    → Item keys: {list(value[0].keys())[:10]}")
                        else:
                            print(f"  {key}: {value}")
                    print()
            else:
                print("\nNo SD-WAN configurations found.")
                print("SD-WAN is used for multi-WAN failover and site-to-site VPN.")
                print("If you have multiple internet providers configured, they would appear here.")

# Check ISP Metrics
if 'Get ISP Metrics' in found_endpoints or 'Query ISP Metrics' in found_endpoints:
    print("\n" + "="*60)
    print("ISP METRICS (Internet Provider Monitoring)")
    print("="*60)
    print("ISP Metrics endpoints are available for monitoring internet provider performance.")
    print("These may require query parameters - see API documentation for details.")

# Extract WAN/HA configuration from hosts data
print("\n" + "="*60)
print("WAN/HA CONFIGURATION IN HOSTS DATA")
print("="*60)

if 'hosts' in dataframes:
    df_hosts = dataframes['hosts']
    
    # Check for WAN data in reportedState.wans
    if 'reportedState.wans' in df_hosts.columns:
        for idx, row in df_hosts.iterrows():
            wans = row.get('reportedState.wans')
            if isinstance(wans, list) and len(wans) > 0:
                print(f"\nHost {idx+1}: {len(wans)} WAN interface(s)")
                for wan in wans:
                    if isinstance(wan, dict):
                        print(f"  WAN: {wan.get('interface', 'unknown')}")
                        print(f"    Type: {wan.get('type', 'unknown')}")
                        print(f"    Enabled: {wan.get('enabled', 'N/A')}")
                        if 'ipv4' in wan and isinstance(wan['ipv4'], dict):
                            print(f"    IPv4: {wan['ipv4'].get('address', 'N/A')}")
                        if 'mac' in wan:
                            print(f"    MAC: {wan.get('mac', 'N/A')}")

print("\n" + "="*60)
print("MANAGEMENT API SUMMARY")
print("="*60)
print("\n✓ Official Site Manager API endpoints tested:")
print("   - SD-WAN Configs: For multi-WAN failover and site-to-site VPN")
print("   - ISP Metrics: For monitoring internet provider performance")
print("   - Note: API is currently READ-ONLY (per official documentation)")
print("\n⚠️  For write/configuration operations (DNS, VPN setup, routing rules):")
print("   Use the local UniFi Network Application API:")
print("   1. Access local controller at https://192.168.1.1:443")
print("   2. Create API token (Settings → API Tokens, requires 2FA)")
print("   3. Query endpoints like:")
print("      - /proxy/network/api/s/{site}/rest/networkconf (networks)")
print("      - /proxy/network/api/s/{site}/rest/vpntunnel (VPN)")
print("      - /proxy/network/api/s/{site}/rest/dynamicdns (DNS)")
print("      - /proxy/network/api/s/{site}/rest/routing (routing/WAN)")
print("\n📚 Full documentation: https://developer.ui.com/site-manager-api/gettingstarted")


## Local Network Application API (Write Operations)

For configuration and write operations (DNS, VPN, routing, firewall rules), you need to access the local UniFi Network Application API using an API token.

**Note:** UniFi OS requires 2FA, so username/password authentication won't work. You must create an API token.


In [ ]:
# Access Local Network Application API for write operations
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

print("="*60)
print("LOCAL NETWORK APPLICATION API ACCESS")
print("="*60)

# Check for API token
local_token = os.getenv('UNIFI_LOCAL_TOKEN')

if not local_token:
    print("\n⚠️  UNIFI_LOCAL_TOKEN not found in ~/.env")
    print("\n" + "="*60)
    print("TO CREATE API TOKEN:")
    print("="*60)
    print("""
1. Log in to UniFi OS at https://192.168.1.1
2. Go to Settings → API Tokens (or User Settings → API Tokens)
3. Click "Create New Token"
4. Give it a name (e.g., "Notebook Access")
5. Set permissions (Network management, etc.)
6. Copy the token immediately (shown only once)
7. Add to ~/.env as: UNIFI_LOCAL_TOKEN=your_token_here
""")
else:
    print(f"✓ UNIFI_LOCAL_TOKEN found")
    
    # Try to get local controller IP from hosts data
    local_ips = ['192.168.1.1', '10.0.0.1', '172.16.0.1']
    
    # If we have hosts data, extract IPs from there
    if 'hosts' in dataframes:
        df_hosts = dataframes['hosts']
        if 'ipAddress' in df_hosts.columns:
            for ip in df_hosts['ipAddress'].dropna():
                if isinstance(ip, str) and any(ip.startswith(prefix) for prefix in ['192.168.', '10.', '172.']):
                    if ip.split('.')[0] not in [ip.split('.')[0] for ip in local_ips]:
                        local_ips.insert(0, ip)
    
    print(f"✓ Testing connection to local controller...\n")
    
    connected = False
    local_session = None
    base_url = None
    site_id = None
    
    for ip in local_ips:
        test_urls = [
            f'https://{ip}:443',
            f'http://{ip}:8080',
        ]
        
        for test_base in test_urls:
            try:
                session = requests.Session()
                session.verify = False
                session.headers.update({
                    'Authorization': f'Bearer {local_token}',
                    'Content-Type': 'application/json'
                })
                
                # Test with sites endpoint
                sites_url = f"{test_base}/proxy/network/api/self/sites"
                resp = session.get(sites_url, timeout=10)
                
                if resp.status_code == 200:
                    data = resp.json()
                    if isinstance(data, dict) and 'data' in data:
                        sites = data['data']
                        print(f"✓ Connected to {test_base}")
                        print(f"✓ Found {len(sites)} site(s)")
                        
                        if len(sites) > 0:
                            site_id = sites[0].get('name', 'default')
                            print(f"✓ Using site: {site_id}")
                            
                            local_session = session
                            base_url = test_base
                            connected = True
                            break
            except requests.exceptions.SSLError:
                # Try HTTP version
                http_base = test_base.replace('https://', 'http://').replace(':443', ':8080')
                try:
                    session = requests.Session()
                    session.headers.update({
                        'Authorization': f'Bearer {local_token}',
                        'Content-Type': 'application/json'
                    })
                    sites_url = f"{http_base}/api/self/sites"
                    resp = session.get(sites_url, timeout=10)
                    if resp.status_code == 200:
                        data = resp.json()
                        if isinstance(data, dict) and 'data' in data:
                            sites = data['data']
                            print(f"✓ Connected to {http_base}")
                            site_id = sites[0].get('name', 'default') if sites else 'default'
                            local_session = session
                            base_url = http_base
                            connected = True
                            break
                except:
                    pass
            except Exception as e:
                continue
        
        if connected:
            break
    
    if connected and local_session and site_id:
        print("\n" + "="*60)
        print("MANAGEMENT ENDPOINTS AVAILABLE")
        print("="*60)
        
        # Test management endpoints
        mgmt_endpoints = {
            'Networks': f'/proxy/network/api/s/{site_id}/rest/networkconf',
            'VPN Tunnels': f'/proxy/network/api/s/{site_id}/rest/vpntunnel',
            'Dynamic DNS': f'/proxy/network/api/s/{site_id}/rest/dynamicdns',
            'Routing': f'/proxy/network/api/s/{site_id}/rest/routing',
            'Firewall Rules': f'/proxy/network/api/s/{site_id}/rest/firewallrule',
            'Settings': f'/proxy/network/api/s/{site_id}/rest/setting',
        }
        
        local_mgmt_data = {}
        
        for name, endpoint in mgmt_endpoints.items():
            try:
                # Adjust endpoint for HTTP if needed
                if base_url.startswith('http://') and endpoint.startswith('/proxy'):
                    endpoint = endpoint.replace('/proxy/network/api', '/api')
                
                url = f"{base_url}{endpoint}"
                resp = local_session.get(url, timeout=10)
                
                if resp.status_code == 200:
                    data = resp.json()
                    if isinstance(data, dict) and 'data' in data:
                        items = data['data']
                        count = len(items) if isinstance(items, list) else 0
                        print(f"✓ {name:20s}: {count} items (READ/WRITE)")
                        local_mgmt_data[name] = items
                    else:
                        print(f"✓ {name:20s}: Available (READ/WRITE)")
                        local_mgmt_data[name] = data
                elif resp.status_code == 401:
                    print(f"✗ {name:20s}: Unauthorized (check token permissions)")
                else:
                    print(f"? {name:20s}: {resp.status_code}")
            except Exception as e:
                print(f"✗ {name:20s}: {str(e)[:40]}")
        
        if local_mgmt_data:
            print(f"\n✓ Successfully accessed {len(local_mgmt_data)} management endpoint(s)")
            print("\n💡 These endpoints support READ and WRITE operations!")
            print("   You can modify DNS, VPN, routing, firewall rules, etc.")
            print("\n   Example: To update a network config:")
            print(f"   PUT {base_url}/proxy/network/api/s/{site_id}/rest/networkconf/{{config_id}}")
            print(f"   Body: {{'name': 'New Network Name', ...}}")
    else:
        print("\n✗ Could not connect to local controller")
        print("   Check that:")
        print("   1. API token is correct")
        print("   2. Controller is accessible at 192.168.1.1 (or your network IP)")
        print("   3. Token has proper permissions")


## Deep Dive: Explore Site-Specific Data

Many UniFi APIs require a site ID. Let's get site IDs from the hosts/sites data and explore site-specific endpoints.


In [10]:
# Extract site IDs from hosts or sites data
site_ids = []

# Try to get sites from hosts DataFrame
if 'hosts' in dataframes:
    df_hosts = dataframes['hosts']
    # Common column names for site IDs
    for col in ['site_id', 'id', 'siteId', 'host_id']:
        if col in df_hosts.columns:
            site_ids = df_hosts[col].dropna().unique().tolist()
            print(f"Found {len(site_ids)} site(s) from 'hosts' DataFrame (column: {col})")
            break

# If no sites found, try sites DataFrame
if not site_ids and 'sites' in dataframes:
    df_sites = dataframes['sites']
    for col in ['site_id', 'id', 'siteId']:
        if col in df_sites.columns:
            site_ids = df_sites[col].dropna().unique().tolist()
            print(f"Found {len(site_ids)} site(s) from 'sites' DataFrame (column: {col})")
            break

# If still no sites, try to find any ID columns
if not site_ids:
    print("Searching for site IDs in all DataFrames...")
    for name, df in dataframes.items():
        id_cols = [col for col in df.columns if 'id' in col.lower()]
        if id_cols:
            for col in id_cols:
                unique_ids = df[col].dropna().unique()
                if len(unique_ids) > 0:
                    print(f"  Found IDs in {name}.{col}: {unique_ids[:5]}")
                    if 'site' in col.lower() or len(unique_ids) <= 10:
                        site_ids = unique_ids.tolist()
                        break

if site_ids:
    print(f"\n✓ Using site IDs: {site_ids[:5]}")
    if len(site_ids) > 5:
        print(f"  ... and {len(site_ids) - 5} more")
else:
    print("\n⚠️  No site IDs found. Some site-specific endpoints may not work.")
    print("   Available DataFrame columns:")
    for name, df in list(dataframes.items())[:3]:
        print(f"   {name}: {list(df.columns)[:10]}")


Found 2 site(s) from 'hosts' DataFrame (column: id)

✓ Using site IDs: ['67be1762-80ee-46dd-a25f-dd69d18da8c6', '74ACB93D0FFB0000000004AEDD420000000004E40634000000005EBA61DE:1546669740']


## Fetch Site-Specific Data

If we have site IDs, let's try to fetch more detailed site-specific data.


In [11]:
# Site-specific endpoints to try
if site_ids:
    site_id = site_ids[0]  # Use first site ID
    print(f"Exploring site-specific endpoints for site: {site_id}\n")
    
    site_endpoints = {
        'site_devices': f'https://api.ui.com/v1/sites/{site_id}/devices',
        'site_clients': f'https://api.ui.com/v1/sites/{site_id}/clients',
        'site_wlans': f'https://api.ui.com/v1/sites/{site_id}/wlans',
        'site_networks': f'https://api.ui.com/v1/sites/{site_id}/networks',
        'site_events': f'https://api.ui.com/v1/sites/{site_id}/events',
        'site_insights': f'https://api.ui.com/v1/sites/{site_id}/insights',
    }
    
    site_data = {}
    
    for name, endpoint in site_endpoints.items():
        try:
            response = api_session.get(endpoint, timeout=15)
            if response.status_code == 200:
                data = response.json()
                site_data[name] = data
                if isinstance(data, dict) and 'data' in data:
                    count = len(data['data']) if isinstance(data['data'], list) else 'N/A'
                    print(f"✓ {name:20s} - {count} items")
                else:
                    print(f"✓ {name:20s} - data retrieved")
            elif response.status_code == 404:
                print(f"✗ {name:20s} - Not found (endpoint may not exist)")
            elif response.status_code == 401:
                print(f"✗ {name:20s} - Unauthorized")
            else:
                print(f"✗ {name:20s} - Error {response.status_code}")
        except Exception as e:
            print(f"✗ {name:20s} - {str(e)[:50]}")
    
    # Create DataFrames from site-specific data
    print(f"\nCreating DataFrames from site-specific data...")
    for name, data in site_data.items():
        df = create_dataframe(data, name)
        if df is not None:
            dataframes[f'site_{name}'] = df
            print(f"✓ Added site_{name} DataFrame ({df.shape[0]} rows)")
    
    print(f"\n✓ Total DataFrames available: {len(dataframes)}")
else:
    print("Skipping site-specific endpoints (no site IDs found)")


Exploring site-specific endpoints for site: 67be1762-80ee-46dd-a25f-dd69d18da8c6

✗ site_devices         - Not found (endpoint may not exist)
✗ site_clients         - Not found (endpoint may not exist)
✗ site_wlans           - Not found (endpoint may not exist)
✗ site_networks        - Not found (endpoint may not exist)
✗ site_events          - Not found (endpoint may not exist)
✗ site_insights        - Not found (endpoint may not exist)

Creating DataFrames from site-specific data...

✓ Total DataFrames available: 3


## Data Analysis & Insights

Let's perform deeper analysis on the available data.


In [12]:
print("="*60)
print("DETAILED DATA ANALYSIS")
print("="*60)
print()

# Helper function to print value counts nicely
def print_value_counts(df, col, label="", max_items=20):
    """Print value counts in a formatted way"""
    if col not in df.columns:
        return
    counts = df[col].value_counts().head(max_items)
    if label:
        print(f"\n{label}:")
    for item, count in counts.items():
        print(f"  {item}: {count}")

# 1. Device Analysis
if 'devices' in dataframes:
    df = dataframes['devices']
    print("📱 DEVICE ANALYSIS")
    print("-" * 60)
    
    # Find relevant columns using vectorized operations
    model_cols = [c for c in df.columns if 'model' in c.lower()]
    status_cols = [c for c in df.columns if any(x in c.lower() for x in ['status', 'state'])]
    version_cols = [c for c in df.columns if any(x in c.lower() for x in ['version', 'firmware'])]
    
    if model_cols:
        print_value_counts(df, model_cols[0], "Device Models")
    
    if status_cols:
        print_value_counts(df, status_cols[0], f"Device Status (column: {status_cols[0]})")
    
    if version_cols:
        print_value_counts(df, version_cols[0], "Firmware Versions", max_items=10)
    
    print()

# 2. Client Analysis
if 'clients' in dataframes:
    df = dataframes['clients']
    print("👥 CLIENT ANALYSIS")
    print("-" * 60)
    print(f"\nTotal Clients: {len(df)}")
    
    if 'essid' in df.columns:
        print_value_counts(df, 'essid', "Clients by SSID")
    
    if 'ip' in df.columns:
        valid_ips = df['ip'].dropna()
        print(f"\nIP Addresses: {len(valid_ips)} clients with IPs")
        if len(valid_ips) > 0:
            print("  Sample IPs:", valid_ips.head(5).tolist())
    
    print()

# 3. Network Analysis
if 'networks' in dataframes:
    df = dataframes['networks']
    print("🌐 NETWORK ANALYSIS")
    print("-" * 60)
    print(f"\nTotal Networks: {len(df)}")
    
    if 'purpose' in df.columns:
        print_value_counts(df, 'purpose', "Networks by Purpose")
    
    # Enabled networks using vectorized operation
    enabled_cols = [c for c in df.columns if 'enabled' in c.lower()]
    if enabled_cols and df[enabled_cols[0]].dtype == bool:
        enabled_count = df[enabled_cols[0]].sum()
        print(f"\nEnabled Networks: {enabled_count} / {len(df)}")
    
    print()

# 4. Events Analysis
if 'events' in dataframes:
    df = dataframes['events']
    print("📋 EVENTS ANALYSIS")
    print("-" * 60)
    print(f"\nTotal Events: {len(df)}")
    
    type_cols = [c for c in df.columns if any(x in c.lower() for x in ['type', 'event'])]
    if type_cols:
        print_value_counts(df, type_cols[0], f"Event Types (column: {type_cols[0]})", max_items=10)
    
    print()


DETAILED DATA ANALYSIS

📱 DEVICE ANALYSIS
------------------------------------------------------------



In [13]:
# Install visualization libraries
import sys
!{sys.executable} -m pip install matplotlib seaborn --quiet

import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('default')
sns.set_palette("husl")

# Create figure with subplots
fig_count = 0

# 1. Device Model Distribution
if 'devices' in dataframes and 'model' in dataframes['devices'].columns:
    fig_count += 1
    plt.figure(fig_count, figsize=(10, 6))
    model_counts = dataframes['devices']['model'].value_counts()
    model_counts.plot(kind='bar')
    plt.title('Device Distribution by Model', fontsize=14, fontweight='bold')
    plt.xlabel('Model', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# 2. Clients by SSID
if 'clients' in dataframes and 'essid' in dataframes['clients'].columns:
    fig_count += 1
    plt.figure(fig_count, figsize=(10, 6))
    ssid_counts = dataframes['clients']['essid'].value_counts()
    ssid_counts.plot(kind='bar')
    plt.title('Clients by SSID', fontsize=14, fontweight='bold')
    plt.xlabel('SSID', fontsize=12)
    plt.ylabel('Number of Clients', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# 3. Network Purpose Distribution
if 'networks' in dataframes and 'purpose' in dataframes['networks'].columns:
    fig_count += 1
    plt.figure(fig_count, figsize=(8, 6))
    purpose_counts = dataframes['networks']['purpose'].value_counts()
    purpose_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title('Networks by Purpose', fontsize=14, fontweight='bold')
    plt.ylabel('')
    plt.tight_layout()
    plt.show()

# 4. Device Status (if available)
if 'devices' in dataframes:
    status_cols = [col for col in dataframes['devices'].columns if 'status' in col.lower() or 'state' in col.lower()]
    if status_cols:
        fig_count += 1
        plt.figure(fig_count, figsize=(8, 6))
        status_counts = dataframes['devices'][status_cols[0]].value_counts()
        status_counts.plot(kind='bar', color=['green', 'orange', 'red', 'gray'])
        plt.title('Device Status Distribution', fontsize=14, fontweight='bold')
        plt.xlabel('Status', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

print(f"\n✓ Created {fig_count} visualization(s)")



✓ Created 0 visualization(s)


## Explore Nested Data Structures

Some API responses may contain nested dictionaries or lists. Let's flatten and explore these.


In [14]:
def flatten_dict(d, parent_key='', sep='_'):
    """
    Flatten nested dictionary
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list) and len(v) > 0 and isinstance(v[0], dict):
            # Handle list of dicts by taking first item
            items.extend(flatten_dict(v[0], new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Check for nested structures in DataFrames
print("Exploring nested data structures...\n")

nested_dataframes = {}

for name, df in dataframes.items():
    # Check if any columns contain dict or list types
    has_nested = False
    for col in df.columns:
        sample = df[col].dropna()
        if len(sample) > 0:
            first_val = sample.iloc[0]
            if isinstance(first_val, dict) or (isinstance(first_val, list) and len(first_val) > 0):
                has_nested = True
                print(f"  {name}: Found nested data in column '{col}'")
                break
    
    if has_nested:
        # Try to flatten a sample row
        try:
            sample_row = df.iloc[0].to_dict()
            flattened = flatten_dict(sample_row)
            if len(flattened) > len(df.columns):
                print(f"    → Can be expanded from {len(df.columns)} to ~{len(flattened)} columns")
        except:
            pass

# Show example of expanding nested data
print("\n" + "="*60)
print("EXAMPLE: Expanding nested device data")
print("="*60)

if 'devices' in dataframes:
    df_devices = dataframes['devices']
    print("\nOriginal columns:", list(df_devices.columns)[:10])
    
    # Show which columns might have nested data
    for col in df_devices.columns[:5]:
        sample = df_devices[col].dropna()
        if len(sample) > 0:
            val = sample.iloc[0]
            if isinstance(val, dict):
                print(f"\nColumn '{col}' contains dict with keys: {list(val.keys())[:10]}")
            elif isinstance(val, list) and len(val) > 0:
                print(f"\nColumn '{col}' contains list with {len(val)} items")
                if isinstance(val[0], dict):
                    print(f"  First item keys: {list(val[0].keys())[:10]}")


Exploring nested data structures...

  hosts: Found nested data in column 'userData.permissions.network.management'
  sites: Found nested data in column 'statistics.internetIssues'
  devices: Found nested data in column 'devices'
    → Can be expanded from 3 to ~24 columns

EXAMPLE: Expanding nested device data

Original columns: ['hostId', 'devices', 'updatedAt']

Column 'devices' contains list with 1 items
  First item keys: ['id', 'mac', 'name', 'model', 'shortname', 'ip', 'productLine', 'status', 'version', 'firmwareStatus']


## Summary Report

Generate a comprehensive summary report of all findings.


In [15]:
from datetime import datetime

print("="*60)
print("UNIFI NETWORK SUMMARY REPORT")
print("="*60)
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# Overall statistics
print("📊 DATA OVERVIEW")
print("-" * 60)
print(f"Total DataFrames: {len(dataframes)}")
print(f"Available DataFrames: {', '.join(sorted(dataframes.keys()))}")
print()

# Count items in each DataFrame
print("📈 DATA COUNTS")
print("-" * 60)
for name in sorted(dataframes.keys()):
    df = dataframes[name]
    print(f"  {name:20s}: {df.shape[0]:5d} rows × {df.shape[1]:3d} columns")

print()

# Key findings
print("🔍 KEY FINDINGS")
print("-" * 60)

# Devices
if 'devices' in dataframes:
    df_devices = dataframes['devices']
    print(f"\n📱 Devices: {len(df_devices)} total")
    if 'model' in df_devices.columns:
        unique_models = df_devices['model'].nunique()
        print(f"   - Unique models: {unique_models}")
    if 'name' in df_devices.columns:
        named_devices = df_devices['name'].notna().sum()
        print(f"   - Devices with names: {named_devices}")

# Clients
if 'clients' in dataframes:
    df_clients = dataframes['clients']
    print(f"\n👥 Clients: {len(df_clients)} total")
    if 'essid' in df_clients.columns:
        unique_ssids = df_clients['essid'].nunique()
        print(f"   - Unique SSIDs: {unique_ssids}")
    if 'ip' in df_clients.columns:
        clients_with_ip = df_clients['ip'].notna().sum()
        print(f"   - Clients with IPs: {clients_with_ip}")

# Networks
if 'networks' in dataframes:
    df_networks = dataframes['networks']
    print(f"\n🌐 Networks: {len(df_networks)} total")
    if 'purpose' in df_networks.columns:
        unique_purposes = df_networks['purpose'].nunique()
        print(f"   - Unique purposes: {unique_purposes}")

# Events
if 'events' in dataframes:
    df_events = dataframes['events']
    print(f"\n📋 Events: {len(df_events)} total")

print()
print("="*60)
print("\n💾 All DataFrames are available in the 'dataframes' dictionary")
print("   Export data using: dataframes['name'].to_csv('filename.csv', index=False)")


UNIFI NETWORK SUMMARY REPORT
Generated: 2025-11-02 20:15:22

📊 DATA OVERVIEW
------------------------------------------------------------
Total DataFrames: 3
Available DataFrames: devices, hosts, sites

📈 DATA COUNTS
------------------------------------------------------------
  devices             :     1 rows ×   3 columns
  hosts               :     2 rows × 151 columns
  sites               :     2 rows ×  43 columns

🔍 KEY FINDINGS
------------------------------------------------------------

📱 Devices: 1 total


💾 All DataFrames are available in the 'dataframes' dictionary
   Export data using: dataframes['name'].to_csv('filename.csv', index=False)


In [16]:
# Additional endpoints to explore
additional_endpoints = [
    'https://api.ui.com/v1/account',
    'https://api.ui.com/v1/subscriptions',
    'https://api.ui.com/v1/notifications',
    'https://api.ui.com/v1/alerts',
    'https://api.ui.com/v1/backups',
    'https://api.ui.com/v1/settings',
    'https://api.ui.com/v1/activity',
    'https://api.ui.com/v1/statistics',
    'https://api.ui.com/v1/traffic',
    'https://api.ui.com/v1/health',
]

print("Trying additional API endpoints...\n")
additional_data = {}

for endpoint in additional_endpoints:
    endpoint_name = endpoint.split('/')[-1]
    try:
        response = api_session.get(endpoint, timeout=10)
        if response.status_code == 200:
            data = response.json()
            additional_data[endpoint_name] = data
            print(f"✓ {endpoint_name:20s} - Available")
        elif response.status_code == 404:
            pass  # Silent - endpoint doesn't exist
        else:
            print(f"✗ {endpoint_name:20s} - Status {response.status_code}")
    except Exception as e:
        pass  # Silent - connection errors

if additional_data:
    print(f"\n✓ Found {len(additional_data)} additional endpoints with data")
    print(f"  Endpoints: {', '.join(additional_data.keys())}")
    
    # Create DataFrames from additional data
    for name, data in additional_data.items():
        df = create_dataframe(data, name)
        if df is not None:
            dataframes[name] = df
            print(f"  ✓ Added '{name}' DataFrame")
else:
    print("\n⚠️  No additional endpoints found (this is normal - not all endpoints are available)")


Trying additional API endpoints...


⚠️  No additional endpoints found (this is normal - not all endpoints are available)


## Time-Based Analysis

If your data has timestamps, let's analyze temporal patterns.


In [17]:
# Look for timestamp columns and analyze temporal patterns
import pandas as pd
from datetime import datetime

print("Searching for timestamp data...\n")

for name, df in dataframes.items():
    # Find columns that might be timestamps
    time_cols = [col for col in df.columns if any(word in col.lower() for word in ['time', 'date', 'created', 'updated', 'last', 'timestamp'])]
    
    if time_cols:
        print(f"📅 {name}: Found timestamp columns - {', '.join(time_cols)}")
        
        for col in time_cols[:2]:  # Check first 2 time columns
            sample = df[col].dropna()
            if len(sample) > 0:
                # Try to parse as datetime
                try:
                    if isinstance(sample.iloc[0], (int, float)):
                        # Might be Unix timestamp (milliseconds or seconds)
                        if sample.iloc[0] > 1e10:
                            # Likely milliseconds
                            df[f'{col}_parsed'] = pd.to_datetime(df[col], unit='ms', errors='coerce')
                        else:
                            # Likely seconds
                            df[f'{col}_parsed'] = pd.to_datetime(df[col], unit='s', errors='coerce')
                    else:
                        df[f'{col}_parsed'] = pd.to_datetime(df[col], errors='coerce')
                    
                    valid_times = df[f'{col}_parsed'].dropna()
                    if len(valid_times) > 0:
                        print(f"  ✓ {col}: {len(valid_times)} valid timestamps")
                        print(f"    Range: {valid_times.min()} to {valid_times.max()}")
                        
                        # Show distribution if many timestamps
                        if len(valid_times) > 10:
                            df[f'{col}_date'] = df[f'{col}_parsed'].dt.date
                            daily_counts = df[f'{col}_date'].value_counts().sort_index()
                            print(f"    Daily distribution (last 5 days):")
                            for date, count in list(daily_counts.items())[-5:]:
                                print(f"      {date}: {count}")
                except Exception as e:
                    pass

print("\n💡 Tip: Use parsed timestamp columns for time-series analysis!")


Searching for timestamp data...

📅 hosts: Found timestamp columns - registrationTime, lastConnectionStateChange, latestBackupTime, reportedState.autoUpdate.includeApplications, reportedState.autoUpdate.preferencesPrompt.unifiOS.applications, reportedState.autoUpdate.preferencesPrompt.unifiOS.defaultSchedule.day, reportedState.autoUpdate.preferencesPrompt.unifiOS.defaultSchedule.frequency, reportedState.autoUpdate.preferencesPrompt.unifiOS.defaultSchedule.hour, reportedState.autoUpdate.preferencesPrompt.unifiOS.firmware, reportedState.autoUpdate.schedule.day, reportedState.autoUpdate.schedule.frequency, reportedState.autoUpdate.schedule.hour, reportedState.deviceStateLastChanged, reportedState.features.deviceList.partialUpdates, reportedState.features.infoApis.firmwareUpdate, reportedState.features.updates.applicationReleaseChannels, reportedState.features.updates.applicationSchedules, reportedState.firmwareUpdate.latestAvailableVersion, reportedState.timezone
  ✓ registrationTime: 1 va

/var/folders/w1/rjr00hgj0txg4sd7_4l617tw0000gn/T/ipykernel_18866/393840987.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[f'{col}_parsed'] = pd.to_datetime(df[col], errors='coerce')


## Search and Filter Examples

Practical examples of searching and filtering your data.


In [18]:
print("="*60)
print("SEARCH AND FILTER EXAMPLES")
print("="*60)
print()

# Example 1: Search for specific devices
if 'devices' in dataframes:
    df = dataframes['devices']
    print("1. Search Devices by Name/Model")
    print("-" * 60)
    
    if 'name' in df.columns:
        print("\nDevices with names containing 'Dream' or 'UDM':")
        matching = df[df['name'].str.contains('Dream|UDM', case=False, na=False)]
        if len(matching) > 0:
            # Use vectorized operations instead of iterating
            result = matching[['name', 'model']].fillna('N/A')
            for name, model in result.values:
                print(f"  - {name} ({model})")
        else:
            print("  No matches found")
    
    if 'model' in df.columns:
        print("\nUnique device models:")
        # Use value_counts instead of manual counting
        model_counts = df['model'].value_counts().head(10)
        print(model_counts.to_string())

# Example 2: Filter clients by SSID
if 'clients' in dataframes:
    df = dataframes['clients']
    print("\n\n2. Filter Clients")
    print("-" * 60)
    
    if 'essid' in df.columns:
        print("\nClients grouped by SSID:")
        # Use groupby instead of iterating
        for ssid, group in df.groupby('essid').head(5).groupby('essid'):
            print(f"\n  SSID: {ssid} ({len(df[df['essid'] == ssid])} clients)")
            if 'hostname' in group.columns:
                hostnames = group['hostname'].dropna().unique()[:5]
                print(f"    Hostnames: {', '.join(hostnames)}")

# Example 3: Network configurations
if 'networks' in dataframes:
    df = dataframes['networks']
    print("\n\n3. Network Configurations")
    print("-" * 60)
    
    if 'purpose' in df.columns:
        print("\nNetworks by purpose:")
        # Use groupby and agg instead of iterating
        grouped = df.groupby('purpose', as_index=False).agg({
            'purpose': 'count',
            **{col: lambda x: ', '.join(x.dropna().head(5).tolist()) 
               for col in df.columns if col == 'name'}
        })
        for _, row in grouped.iterrows():
            print(f"\n  {row['purpose']}: network(s)")
            if 'name' in row.index:
                print(f"    Names: {row.get('name', 'N/A')}")

print("\n" + "="*60)


SEARCH AND FILTER EXAMPLES

1. Search Devices by Name/Model
------------------------------------------------------------



## Advanced Queries

More complex queries and aggregations.


In [27]:
df

,hostId,devices,updatedAt,updatedAt_parsed
0,74ACB93D0FFB0000000004AEDD420000000004E4063400...,"[{'id': '74ACB93D0FFB', 'mac': '74ACB93D0FFB',...",2025-10-23T22:31:52Z,2025-10-23 22:31:52+00:00


## Export All Data for Further Analysis

Export all DataFrames to various formats for external analysis tools.


In [19]:
print("="*60)
print("ADVANCED QUERIES AND AGGREGATIONS")
print("="*60)
print()

# 1. Cross-join analysis (if possible)
print("1. Cross-DataFrame Analysis")
print("-" * 60)

# Devices and their IPs - use vectorized operations
if 'devices' in dataframes:
    df = dataframes['devices']
    if 'ip' in df.columns:
        device_ips = df['ip'].dropna().nunique()
        print(f"\nDevice IPs: {device_ips} unique IPs")
        if all(col in df.columns for col in ['name', 'ip', 'model']):
            print("\nDevices with IPs:")
            device_info = df[['name', 'ip', 'model']].dropna(subset=['ip']).head(10)
            # Use apply or to_string instead of iterating
            print(device_info.to_string(index=False))

# Client and device correlation - use set operations
if 'clients' in dataframes and 'devices' in dataframes:
    client_ips = set(dataframes['clients']['ip'].dropna().unique()) if 'ip' in dataframes['clients'].columns else set()
    device_ips = set(dataframes['devices']['ip'].dropna().unique()) if 'ip' in dataframes['devices'].columns else set()
    
    if client_ips or device_ips:
        print("\n\n2. IP Address Analysis")
        print("-" * 60)
        print(f"Client IPs: {len(client_ips)}")
        print(f"Device IPs: {len(device_ips)}")
        
        overlapping = client_ips & device_ips  # Set intersection
        if overlapping:
            print(f"Overlapping IPs (clients that might be devices): {len(overlapping)}")
            print(f"  Sample: {list(overlapping)[:5]}")

# 3. Summary statistics - use describe() directly
print("\n\n3. Summary Statistics")
print("-" * 60)

for name, df in dataframes.items():
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    if len(numeric_cols) > 0:
        print(f"\n{name} - Numeric columns:")
        # Use describe() directly instead of manual stats
        stats = df[numeric_cols[:5]].describe().T[['mean', 'min', 'max']]
        print(stats.to_string())

print("\n" + "="*60)


ADVANCED QUERIES AND AGGREGATIONS

1. Cross-DataFrame Analysis
------------------------------------------------------------


3. Summary Statistics
------------------------------------------------------------

hosts - Numeric columns:
                                                                           mean     min      max
reportedState.firmware_version                                              NaN     NaN      NaN
reportedState.host_type                                                 29960.5     0.0  59921.0
reportedState.inform_port                                                8080.0  8080.0   8080.0
reportedState.mgmt_port                                                  4443.0   443.0   8443.0
reportedState.autoUpdate.preferencesPrompt.unifiOS.defaultSchedule.day      0.0     0.0      0.0

sites - Numeric columns:
                                        mean  min  max
statistics.counts.criticalNotification   0.0  0.0  0.0
statistics.counts.gatewayDevice          0.5  0

In [20]:
from datetime import datetime
import json

# Create comprehensive export
export_dir = "unifi_comprehensive_export"
os.makedirs(export_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print(f"Exporting all data to '{export_dir}'...\n")

# 1. Export all DataFrames to CSV
csv_count = 0
for name, df in dataframes.items():
    csv_file = f"{export_dir}/{name}_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    csv_count += 1
    print(f"✓ CSV: {name:20s} → {csv_file}")

# 2. Export all DataFrames to Excel (one workbook with multiple sheets)
try:
    excel_file = f"{export_dir}/unifi_all_data_{timestamp}.xlsx"
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        for name, df in dataframes.items():
            # Excel sheet names have limitations
            sheet_name = name[:31]  # Excel sheet name limit
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"\n✓ Excel: All DataFrames → {excel_file}")
except ImportError:
    print("\n⚠️  openpyxl not installed - skipping Excel export")
    print("   Install with: pip install openpyxl")
except Exception as e:
    print(f"\n✗ Excel export failed: {e}")

# 3. Export summary metadata as JSON
metadata = {
    'export_timestamp': timestamp,
    'total_dataframes': len(dataframes),
    'dataframes': {}
}

for name, df in dataframes.items():
    metadata['dataframes'][name] = {
        'rows': int(df.shape[0]),
        'columns': int(df.shape[1]),
        'column_names': df.columns.tolist(),
        'dtypes': {str(k): str(v) for k, v in df.dtypes.items()}
    }

metadata_file = f"{export_dir}/metadata_{timestamp}.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✓ Metadata: → {metadata_file}")

print(f"\n{'='*60}")
print(f"✓ Export complete!")
print(f"   - {csv_count} CSV files")
print(f"   - 1 Excel workbook (if available)")
print(f"   - 1 metadata JSON file")
print(f"   Location: {os.path.abspath(export_dir)}")
print(f"{'='*60}")


Exporting all data to 'unifi_comprehensive_export'...

✓ CSV: hosts                → unifi_comprehensive_export/hosts_20251102_201523.csv
✓ CSV: sites                → unifi_comprehensive_export/sites_20251102_201523.csv
✓ CSV: devices              → unifi_comprehensive_export/devices_20251102_201523.csv

⚠️  openpyxl not installed - skipping Excel export
   Install with: pip install openpyxl
✓ Metadata: → unifi_comprehensive_export/metadata_20251102_201523.json

✓ Export complete!
   - 3 CSV files
   - 1 Excel workbook (if available)
   - 1 metadata JSON file
   Location: /Users/lou/Downloads/mid-linux-container-recipe.zurich-07-01-2025__patch2-09-24-2025_10-12-2025_0904.linux.x86-64/unifi_comprehensive_export


## Quick Reference

All DataFrames are stored in the `dataframes` dictionary. Quick access guide:


In [21]:
print("="*60)
print("QUICK REFERENCE GUIDE")
print("="*60)
print()

# Use list comprehension and string formatting more efficiently
key_cols_template = ['name', 'id', 'ip', 'mac', 'model', 'hostname', 'essid']

dataframe_info = [
    (name, df, [col for col in key_cols_template if col in df.columns])
    for name, df in sorted(dataframes.items())
]

print("📊 Available DataFrames:")
for name, df, key_cols in dataframe_info:
    print(f"  • dataframes['{name}']")
    print(f"    → {df.shape[0]} rows × {df.shape[1]} columns")
    if key_cols:
        print(f"    → Key columns: {', '.join(key_cols[:5])}")
    print()

commands = {
    "View all DataFrames": "list(dataframes.keys())",
    "Access a DataFrame": "df = dataframes['devices']",
    "View DataFrame": "dataframes['devices'].head()",
    "Filter data": "dataframes['devices'][dataframes['devices']['model'] == 'UDM']",
    "Group by": "dataframes['clients'].groupby('essid').size()",
    "Export": "dataframes['devices'].to_csv('devices.csv', index=False)"
}

print("\n💡 Useful Commands:")
for desc, cmd in commands.items():
    print(f"  # {desc}")
    print(f"  {cmd}\n")

print("="*60)
print("Happy exploring! 🚀")
print("="*60)


QUICK REFERENCE GUIDE

📊 Available DataFrames:
  • dataframes['devices']
    → 1 rows × 4 columns

  • dataframes['hosts']
    → 2 rows × 153 columns
    → Key columns: id

  • dataframes['sites']
    → 2 rows × 45 columns


💡 Useful Commands:
  # View all DataFrames
  list(dataframes.keys())

  # Access a DataFrame
  df = dataframes['devices']

  # View DataFrame
  dataframes['devices'].head()

  # Filter data
  dataframes['devices'][dataframes['devices']['model'] == 'UDM']

  # Group by
  dataframes['clients'].groupby('essid').size()

  # Export
  dataframes['devices'].to_csv('devices.csv', index=False)

Happy exploring! 🚀


## Infer Relational Schema from Data

Analyze the actual data structure to infer a normalized relational schema with relationships.


In [22]:
from collections import defaultdict
import json

print("="*60)
print("SCHEMA INFERENCE FROM ACTUAL DATA")
print("="*60)
print()

# Analyze each DataFrame structure
schema_info = {}

for name, df in dataframes.items():
    print(f"\n{'='*60}")
    print(f"Analyzing: {name.upper()}")
    print(f"{'='*60}")
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")
    
    # Identify potential keys
    id_cols = [c for c in df.columns if 'id' in c.lower() and c.lower() not in ['device', 'network']]
    unique_cols = []
    foreign_key_candidates = []
    
    # Check for unique identifiers (skip unhashable types like lists/dicts)
    for col in df.columns:
        try:
            # Check if column has hashable values (not lists/dicts)
            sample = df[col].dropna()
            if len(sample) > 0:
                first_val = sample.iloc[0]
                if isinstance(first_val, (list, dict)):
                    continue  # Skip unhashable types
            # Now safe to check nunique
            if df[col].nunique() == len(df) and df[col].notna().sum() > 0:
                unique_cols.append(col)
        except (TypeError, ValueError):
            # Skip columns that cause errors with nunique
            continue
    
    # Check for potential foreign keys (values that appear in other tables)
    for col in df.columns:
        try:
            # Skip unhashable types
            sample = df[col].dropna()
            if len(sample) > 0:
                first_val = sample.iloc[0]
                if isinstance(first_val, (list, dict)):
                    continue
            # Check if column name suggests FK and is not unique
            if any(keyword in col.lower() for keyword in ['id', 'host', 'site', 'device', 'network']):
                if df[col].nunique() < len(df):  # Not unique = might be FK
                    foreign_key_candidates.append(col)
        except (TypeError, ValueError):
            continue
    
    # Analyze data types
    data_types = {
        'categorical': list(df.select_dtypes(include=['object', 'bool']).columns),
        'numeric': list(df.select_dtypes(include=['int64', 'float64']).columns),
        'datetime': [c for c in df.columns if 'time' in c.lower() or 'date' in c.lower()]
    }
    
    # Look for nested structures (dict/list columns)
    nested_cols = []
    for col in df.columns:
        sample = df[col].dropna()
        if len(sample) > 0:
            val = sample.iloc[0]
            if isinstance(val, (dict, list)):
                nested_cols.append((col, type(val).__name__))
    
    schema_info[name] = {
        'columns': list(df.columns),
        'row_count': len(df),
        'id_cols': id_cols,
        'unique_cols': unique_cols,
        'foreign_key_candidates': foreign_key_candidates,
        'data_types': data_types,
        'nested_cols': nested_cols,
        'sample_data': df.head(1).to_dict('records')[0] if len(df) > 0 else {}
    }
    
    print(f"Primary Key Candidates: {unique_cols if unique_cols else 'None found (might need composite)'}")
    print(f"Foreign Key Candidates: {foreign_key_candidates if foreign_key_candidates else 'None'}")
    print(f"Nested Columns: {[c[0] for c in nested_cols] if nested_cols else 'None'}")
    
    # Show column summary
    print(f"\nColumn Summary:")
    for col in df.columns[:15]:
        dtype = str(df[col].dtype)
        non_null = df[col].notna().sum()
        try:
            # Check if column has hashable values
            sample = df[col].dropna()
            if len(sample) > 0:
                first_val = sample.iloc[0]
                if isinstance(first_val, (list, dict)):
                    unique = "N/A (list/dict)"
                else:
                    unique = df[col].nunique()
            else:
                unique = 0
        except (TypeError, ValueError):
            unique = "N/A"
        print(f"  {col:30s} | {dtype:10s} | {non_null:4d}/{len(df)} non-null | {unique} unique")

print(f"\n{'='*60}")
print(f"ANALYZED {len(schema_info)} DATA TABLES")
print(f"{'='*60}")


SCHEMA INFERENCE FROM ACTUAL DATA


Analyzing: HOSTS
Shape: 2 rows × 153 columns

Primary Key Candidates: ['id', 'hardwareId', 'type', 'registrationTime', 'lastConnectionStateChange', 'latestBackupTime', 'reportedState.controller_uuid', 'reportedState.host_type', 'reportedState.hostname', 'reportedState.mgmt_port', 'reportedState.name', 'reportedState.state', 'reportedState.version', 'lastConnectionStateChange_parsed']
Foreign Key Candidates: ['reportedState.hardware_id', 'reportedState.override_inform_host', 'userData.features.deviceGroups', 'userData.localId', 'userData.roleId', 'reportedState.anonid', 'reportedState.deviceErrorCode', 'reportedState.deviceState', 'reportedState.deviceStateLastChanged', 'reportedState.features.deviceList.autolinkDevices', 'reportedState.features.deviceList.partialUpdates', 'reportedState.features.deviceList.ucp4Events', 'reportedState.features.identity.hostingIdentityHubScore', 'reportedState.features.identity.standard', 'reportedState.features.identi

## Proposed Relational Schema

Based on the analysis, here's the inferred relational schema with relationships.


In [23]:
# Map relationships between tables
print("="*60)
print("RELATIONAL SCHEMA")
print("="*60)
print()

# Find relationships by matching column names across tables
relationships = defaultdict(list)

for table1_name, table1_info in schema_info.items():
    df1 = dataframes[table1_name]
    for table2_name, table2_info in schema_info.items():
        if table1_name == table2_name:
            continue
        
        # Check if any columns match (potential FK relationships)
        for col1 in table1_info['columns']:
            for col2 in table2_info['columns']:
                if col1.lower() == col2.lower():
                    # Check if values actually overlap
                    if col1 in df1.columns and col2 in dataframes[table2_name].columns:
                        try:
                            # Skip unhashable types
                            sample1 = df1[col1].dropna()
                            sample2 = dataframes[table2_name][col2].dropna()
                            if len(sample1) > 0 and len(sample2) > 0:
                                if isinstance(sample1.iloc[0], (list, dict)) or isinstance(sample2.iloc[0], (list, dict)):
                                    continue
                            vals1 = set(df1[col1].dropna().unique())
                            vals2 = set(dataframes[table2_name][col2].dropna().unique())
                            overlap = vals1 & vals2
                            if len(overlap) > 0 and len(vals1) <= len(vals2):
                                # Likely FK: table1.col1 -> table2.col2
                                relationships[table1_name].append({
                                    'foreign_key': col1,
                                    'references': table2_name,
                                    'referenced_column': col2,
                                    'overlap_count': len(overlap)
                                })
                        except (TypeError, ValueError):
                            # Skip columns that can't be hashed/compared
                            continue

# Define schema for each table
print("TABLES AND COLUMNS:\n")
for table_name, info in schema_info.items():
    print(f"📊 {table_name.upper()}")
    print("-" * 60)
    
    # Primary key
    pk = info['unique_cols'][0] if info['unique_cols'] else info['id_cols'][0] if info['id_cols'] else None
    if pk:
        print(f"  PRIMARY KEY: {pk}")
    else:
        print(f"  PRIMARY KEY: (composite or none identified)")
    
    # Foreign keys
    fks = relationships.get(table_name, [])
    if fks:
        print(f"  FOREIGN KEYS:")
        for fk_info in fks:
            print(f"    {fk_info['foreign_key']} -> {fk_info['references']}.{fk_info['referenced_column']} ({fk_info['overlap_count']} matching values)")
    
    # Columns grouped by type
    print(f"\n  COLUMNS:")
    df = dataframes[table_name]
    for col in info['columns']:
        dtype = df[col].dtype
        nullable = "NULL" if df[col].isna().any() else "NOT NULL"
        
        # Check if it's a FK
        is_fk = any(fk['foreign_key'] == col for fk in fks)
        fk_marker = " [FK]" if is_fk else ""
        pk_marker = " [PK]" if col == pk else ""
        
        print(f"    {col:30s} {str(dtype):15s} {nullable:10s}{pk_marker}{fk_marker}")
    
    print()

# Entity Relationship Summary
print(f"\n{'='*60}")
print("ENTITY RELATIONSHIPS")
print(f"{'='*60}\n")

if relationships:
    for table, fks in relationships.items():
        if fks:
            print(f"{table}:")
            for fk in fks:
                print(f"  → {fk['references']} (via {fk['foreign_key']})")
            print()
else:
    print("No explicit relationships found in column names.")
    print("Relationships may be implicit through nested structures.")
    
print(f"\n💡 This schema can be used to create SQL tables or a proper database!")


RELATIONAL SCHEMA

TABLES AND COLUMNS:

📊 HOSTS
------------------------------------------------------------
  PRIMARY KEY: id

  COLUMNS:
    id                             object          NOT NULL   [PK]
    hardwareId                     object          NOT NULL  
    type                           object          NOT NULL  
    ipAddress                      object          NOT NULL  
    owner                          bool            NOT NULL  
    isBlocked                      bool            NOT NULL  
    registrationTime               object          NOT NULL  
    lastConnectionStateChange      object          NOT NULL  
    latestBackupTime               object          NOT NULL  
    userData.permissions.network.management object          NOT NULL  
    userData.status                object          NOT NULL  
    reportedState.controller_uuid  object          NOT NULL  
    reportedState.firmware_version float64         NULL      
    reportedState.hardware_id      object

## Generate SQL CREATE Statements

Create SQL DDL statements based on the inferred schema.


In [24]:
# SQL type mapping
def pandas_to_sql_type(dtype_str):
    """Map pandas dtypes to SQL types"""
    dtype_lower = str(dtype_str).lower()
    if 'int' in dtype_lower:
        return 'INTEGER'
    elif 'float' in dtype_lower:
        return 'REAL'
    elif 'bool' in dtype_lower:
        return 'BOOLEAN'
    elif 'datetime' in dtype_lower or 'timestamp' in dtype_lower:
        return 'TIMESTAMP'
    else:
        return 'TEXT'

print("="*60)
print("SQL CREATE TABLE STATEMENTS")
print("="*60)
print()

sql_statements = []

for table_name, info in schema_info.items():
    df = dataframes[table_name]
    
    # Get primary key
    pk = info['unique_cols'][0] if info['unique_cols'] else info['id_cols'][0] if info['id_cols'] else None
    
    # Get foreign keys
    fks = relationships.get(table_name, [])
    fk_cols = {fk['foreign_key']: fk for fk in fks}
    
    # Build CREATE TABLE statement
    table_name_sql = table_name.replace('-', '_').replace(' ', '_')
    sql = f"CREATE TABLE {table_name_sql} (\n"
    
    columns_def = []
    for col in info['columns']:
        # Skip nested columns for now (would need separate tables)
        if col in [c[0] for c in info['nested_cols']]:
            continue
            
        sql_type = pandas_to_sql_type(df[col].dtype)
        nullable = "" if df[col].isna().any() else " NOT NULL"
        
        col_def = f"    {col} {sql_type}{nullable}"
        
        # Add primary key constraint
        if col == pk:
            col_def += " PRIMARY KEY"
        
        columns_def.append(col_def)
    
    sql += ",\n".join(columns_def)
    
    # Add foreign key constraints
    if fks:
        sql += ",\n"
        fk_constraints = []
        for fk_info in fks:
            ref_table = fk_info['references'].replace('-', '_').replace(' ', '_')
            ref_col = fk_info['referenced_column']
            fk_constraints.append(
                f"    FOREIGN KEY ({fk_info['foreign_key']}) REFERENCES {ref_table}({ref_col})"
            )
        sql += ",\n".join(fk_constraints)
    
    sql += "\n);"
    
    sql_statements.append(sql)
    print(sql)
    print()

# Save to file
if sql_statements:
    sql_file = "unifi_schema.sql"
    with open(sql_file, 'w') as f:
        f.write("-- UniFi Database Schema\n")
        f.write("-- Generated from API data analysis\n")
        f.write(f"-- {len(sql_statements)} tables\n\n")
        f.write("\n\n".join(sql_statements))
    
    print(f"\n✓ SQL schema saved to: {sql_file}")
    print(f"   {len(sql_statements)} CREATE TABLE statements")


SQL CREATE TABLE STATEMENTS

CREATE TABLE hosts (
    id TEXT NOT NULL PRIMARY KEY,
    hardwareId TEXT NOT NULL,
    type TEXT NOT NULL,
    ipAddress TEXT NOT NULL,
    owner BOOLEAN NOT NULL,
    isBlocked BOOLEAN NOT NULL,
    registrationTime TEXT NOT NULL,
    lastConnectionStateChange TEXT NOT NULL,
    latestBackupTime TEXT NOT NULL,
    userData.status TEXT NOT NULL,
    reportedState.controller_uuid TEXT NOT NULL,
    reportedState.firmware_version REAL,
    reportedState.hardware_id TEXT,
    reportedState.host_type INTEGER NOT NULL,
    reportedState.hostname TEXT NOT NULL,
    reportedState.inform_port REAL,
    reportedState.mgmt_port INTEGER NOT NULL,
    reportedState.name TEXT NOT NULL,
    reportedState.override_inform_host TEXT,
    reportedState.release_channel TEXT,
    reportedState.state TEXT NOT NULL,
    reportedState.version TEXT NOT NULL,
    userData.email TEXT,
    userData.features.deviceGroups TEXT,
    userData.features.floorplan.canEdit TEXT,
    userDa

## Entity Relationship Diagram (ERD)

Visual representation of the relational schema and relationships.


In [25]:
# Generate Mermaid ERD
from IPython.display import Markdown, display
from collections import defaultdict

def sanitize_name(name):
    """Sanitize names for Mermaid (remove special chars, spaces)"""
    return name.replace(' ', '_').replace('-', '_').replace('.', '_')

def sanitize_column_name(col_name):
    """Sanitize column names - Mermaid can handle most but escape quotes"""
    return col_name.replace('"', '\\"')

# Ensure relationships is defined (in case schema analysis wasn't run)
if 'relationships' not in globals():
    relationships = defaultdict(list)
    # Quick relationship detection if schema_info exists
    if 'schema_info' in globals():
        for table1_name, table1_info in schema_info.items():
            df1 = dataframes[table1_name]
            for table2_name, table2_info in schema_info.items():
                if table1_name == table2_name:
                    continue
                for col1 in table1_info['columns']:
                    for col2 in table2_info['columns']:
                        if col1.lower() == col2.lower():
                            if col1 in df1.columns and col2 in dataframes[table2_name].columns:
                                try:
                                    # Skip unhashable types
                                    sample1 = df1[col1].dropna()
                                    sample2 = dataframes[table2_name][col2].dropna()
                                    if len(sample1) > 0 and len(sample2) > 0:
                                        if isinstance(sample1.iloc[0], (list, dict)) or isinstance(sample2.iloc[0], (list, dict)):
                                            continue
                                    vals1 = set(df1[col1].dropna().unique())
                                    vals2 = set(dataframes[table2_name][col2].dropna().unique())
                                    overlap = vals1 & vals2
                                    if len(overlap) > 0 and len(vals1) <= len(vals2):
                                        relationships[table1_name].append({
                                            'foreign_key': col1,
                                            'references': table2_name,
                                            'referenced_column': col2,
                                            'overlap_count': len(overlap)
                                        })
                                except (TypeError, ValueError):
                                    continue

# Build Mermaid ERD syntax
mermaid_code = "erDiagram\n"

# Add entities (tables)
for table_name, info in schema_info.items():
    table_name_clean = sanitize_name(table_name)
    pk = info['unique_cols'][0] if info['unique_cols'] else info['id_cols'][0] if info['id_cols'] else None
    
    # Get visible columns (exclude nested)
    visible_cols = [c for c in info['columns'] if c not in [nc[0] for nc in info['nested_cols']]]
    
    # Add table definition with key columns
    mermaid_code += f"    {table_name_clean} {{\n"
    
    # Show primary key first
    if pk and pk in visible_cols:
        pk_clean = sanitize_column_name(pk)
        mermaid_code += f"        {pk_clean} string PK\n"
    
    # Show other key columns
    key_cols = [c for c in visible_cols if any(x in c.lower() for x in ['id', 'name', 'type', 'status', 'ip', 'mac'])]
    df = dataframes[table_name]
    
    for col in key_cols[:8]:  # Limit columns shown
        if col == pk:
            continue
        # Determine type from pandas dtype
        if col in df.select_dtypes(include=['int64', 'float64']).columns:
            col_type = "number"
        elif 'time' in col.lower() or 'date' in col.lower():
            col_type = "datetime"
        else:
            col_type = "string"
        
        col_clean = sanitize_column_name(col)
        mermaid_code += f"        {col_clean} {col_type}\n"
    
    # Add count indicator if more columns exist
    remaining = len(visible_cols) - len(key_cols) - (1 if pk else 0)
    if remaining > 0:
        mermaid_code += f"        ... {remaining} more columns\n"
    
    mermaid_code += f"    }}\n\n"

# Add relationships
for table_name, fks in relationships.items():
    for fk in fks:
        table1 = sanitize_name(table_name)
        table2 = sanitize_name(fk['references'])
        fk_name = sanitize_column_name(fk['foreign_key'])
        # Many-to-one relationship (FK implies many on the FK side)
        mermaid_code += f"    {table1} ||--o{{ {table2} : \"{fk_name}\"\n"

# Display Mermaid diagram
display(Markdown(f"""```mermaid
{mermaid_code}
```"""))

print(f"\n✓ ERD Generated ({len(schema_info)} tables, {sum(len(v) for v in relationships.values())} relationships)")


```mermaid
erDiagram
    hosts {
        id string PK
        hardwareId string
        type string
        ipAddress string
        userData.status string
        reportedState.controller_uuid string
        reportedState.hardware_id string
        reportedState.host_type number
        ... 85 more columns
    }

    sites {
        siteId string PK
        hostId string
        meta.name string
        statistics.counts.offlineWifiDevice number
        statistics.counts.wifiDevice number
        meta.gatewayMac string
        statistics.gateway.hardwareId string
        statistics.gateway.ipsMode string
        ... 28 more columns
    }

    devices {
        hostId string PK
        ... 1 more columns
    }

    devices ||--o{ sites : "hostId"

```


✓ ERD Generated (3 tables, 1 relationships)


## Alternative: Export Mermaid Code

Export the Mermaid ERD code to a file for use in other tools.


In [26]:
# Export Mermaid code to file (reuse code from above if it exists)
if 'mermaid_code' in globals():
    mermaid_file = "unifi_erd.mmd"
    with open(mermaid_file, 'w') as f:
        f.write(mermaid_code)
    
    print(f"✓ Mermaid ERD saved to: {mermaid_file}")
    print(f"   You can view this at: https://mermaid.live/")
    print(f"   Or use in GitHub/GitLab markdown, Obsidian, or other Mermaid-compatible tools")
    print(f"\nPreview (first 500 chars):")
    print("="*60)
    print(mermaid_code[:500] + "..." if len(mermaid_code) > 500 else mermaid_code)
else:
    print("⚠️  Run the ERD generation cell above first!")


✓ Mermaid ERD saved to: unifi_erd.mmd
   You can view this at: https://mermaid.live/
   Or use in GitHub/GitLab markdown, Obsidian, or other Mermaid-compatible tools

Preview (first 500 chars):
erDiagram
    hosts {
        id string PK
        hardwareId string
        type string
        ipAddress string
        userData.status string
        reportedState.controller_uuid string
        reportedState.hardware_id string
        reportedState.host_type number
        ... 85 more columns
    }

    sites {
        siteId string PK
        hostId string
        meta.name string
        statistics.counts.offlineWifiDevice number
        statistics.counts.wifiDevice number
        meta.gat...


## Debugging: Launch Notebook in Browser

**For better visualization and debugging**, launch this notebook in a browser with Playwright automation hooks. The IDE's notebook viewer is horrible for visualizations - this gives you full browser debugging capabilities!


In [ ]:
# Launch this notebook in browser for better visualization & debugging
from playwright.sync_api import sync_playwright
import subprocess
import time
import sys
from pathlib import Path

notebook_file = Path("unifi_data_analysis.ipynb")
notebook_name = notebook_file.name

print("="*60)
print("LAUNCHING NOTEBOOK IN BROWSER FOR DEBUGGING")
print("="*60)

# Check for Jupyter server
jupyter_url = None
try:
    import requests
    for port in [8888, 8889, 8890]:
        try:
            response = requests.get(f'http://localhost:{port}', timeout=2)
            if response.status_code == 200:
                jupyter_url = f"http://localhost:{port}"
                print(f"✓ Jupyter server running on {jupyter_url}")
                break
        except:
            pass
except:
    pass

if not jupyter_url:
    print("\n🚀 Starting Jupyter server...")
    jupyter_cmd = [sys.executable, '-m', 'jupyter', 'notebook', '--no-browser', '--port=8888']
    subprocess.Popen(jupyter_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("   Waiting for server...")
    for i in range(15):
        time.sleep(1)
        try:
            import requests
            response = requests.get('http://localhost:8888', timeout=1)
            if response.status_code == 200:
                jupyter_url = "http://localhost:8888"
                print(f"✓ Server started on {jupyter_url}")
                break
        except:
            if i % 3 == 0:
                print(f"   Still waiting... ({i+1}/15)")
    if not jupyter_url:
        jupyter_url = "http://localhost:8888"
        time.sleep(2)

if jupyter_url:
    notebook_url = f"{jupyter_url}/notebooks/{notebook_name}"
    print(f"\n🌐 Launching with Playwright browser automation...")
    print(f"   URL: {notebook_url}\n")
    
    try:
        with sync_playwright() as p:
            print("🚀 Launching browser with automation hooks...")
            browser = p.chromium.launch(headless=False, args=['--start-maximized'])
            context = browser.new_context(viewport={'width': 1920, 'height': 1080})
            page = context.new_page()
            
            print("📓 Loading notebook...")
            page.goto(notebook_url, wait_until='networkidle', timeout=30000)
            
            print("✓ Notebook opened in browser!")
            print("\n✅ Browser window open - you can:")
            print("   📊 View all visualizations clearly")
            print("   🐛 Debug with browser developer tools")
            print("   🎯 Step through cells interactively")
            print("   🔧 Use Playwright hooks for automation")
            print(f"\n📝 Server: {jupyter_url}")
            print("💡 Close browser window when done")
            print("\n⏸️  This is your debugging meta-document!\n")
            
            try:
                while browser.connected:
                    time.sleep(1)
            except KeyboardInterrupt:
                print("\n⚠️  Closing browser...")
            
            if browser.connected:
                browser.close()
            print("✓ Browser closed")
            
    except Exception as e:
        print(f"⚠️  Error: {e}")
        import webbrowser
        webbrowser.open(notebook_url)
        print("✓ Opened in default browser")
